In [69]:
import subprocess
import sys

# Function to install packages
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install packages
install('pandas')
install('pyodbc')

import pandas as pd
import pyodbc
import os.path

# SQL Server connection string
sql_conn_string = 'DRIVER={ODBC Driver 17 for SQL Server};   \
                           SERVER=XXXXX-YY\SQL2019;          \
                           DATABASE=Survey_Sample_A18;       \
                           Trusted_Connection=yes'

In [70]:
def strQueryTemplateForAnswerColumn(SURVEY_ID, QUESTION_ID):
    """Generates a query (using the input QUESTION_ID) which will be used 
       to fetch the answers to the input question"""
    result =  'COALESCE( (                                               \
                            SELECT a.Answer_Value                        \
                            FROM Answer as a                             \
                            WHERE                                        \
                                a.UserId = u.UserId                      \
                                AND a.SurveyId = '+str(SURVEY_ID)+'      \
                                AND a.QuestionId = '+str(QUESTION_ID)+'  \
                          )                                              \
                    , -1) AS ANS_Q'+str(QUESTION_ID)
    return result

def strQueryTemplateForNullColumnn(QUESTION_ID):
    """Concatenates a default string for null answers and a question Id"""
    return 'NULL AS ANS_Q'+str(QUESTION_ID)

def strQueryTemplateOuterUnionQuery(SURVEY_ID, DYNAMIC_QUESTION_ANSWERS):
    """Generates query using survey ID and dynamically generated question_answers"""

    result =  'SELECT UserId                                                    \
                      ,'+str(SURVEY_ID)+' as SurveyId                           \
                      ,'+str(DYNAMIC_QUESTION_ANSWERS)+'                        \
               FROM                                                             \
                     [User] as u                                                \
               WHERE EXISTS                                                     \
                (                                                               \
                    SELECT *                                                    \
                    FROM Answer as a                                            \
                    WHERE u.UserId = a.UserId                                   \
                    AND a.SurveyId = '+str(SURVEY_ID)+'                         \
                )'
    
    return result

In [71]:
querySurveys = "SELECT SurveyId     \
                FROM Survey         \
                ORDER BY SurveyId"  

querySurveyStructure =  "SELECT *              \
                         FROM SurveyStructure   \
                         ORDER BY SurveyId"  

def queryQuestionsinSurvey(SURVEY_ID):
    """Takes as input a survey ID and returns a query which will be used to fetch 
       the result set : SurveyId, QuestionId and flag InSurvey indicating whether
       each question is in the survey structure or not """
    output = "SELECT *                          \
              FROM                              \
                   (                            \
                        SELECT                  \
                            SurveyId,           \
                            QuestionId,         \
                            1 as InSurvey       \
                        FROM                    \
                            SurveyStructure     \
                        WHERE                   \
                            SurveyId = "+str(SURVEY_ID)+"    \
                        UNION                                \
                        SELECT                               \
                            "+str(SURVEY_ID)+" as SurveyId,  \
                            Q.QuestionId,                    \
                            0 as InSurvey                    \
                        FROM                                 \
                            Question as Q                    \
                        WHERE NOT EXISTS                     \
                        (                                    \
                            SELECT *                         \
                            FROM SurveyStructure as S        \
                            WHERE S.SurveyId = "+str(SURVEY_ID)+" AND S.QuestionId = Q.QuestionId   \
                        )                                   \
                    ) as t                                  \
                    ORDER BY QuestionId;"
    return output    

In [72]:
def GetAllSurveyDataSQL(sql_conn):
    """ This function mimics a stored procedure in SQL Server. 
        It generates and returns a SQL query string to be used 
        to query for each user, corresponding surveys and answer_questions"""
    
    strFinalQuery = ''
    
    # Fetch the surveys from the database 
    surveys = pd.read_sql(querySurveys, sql_conn)
    
    surveyCount = len(surveys)
    
    # Loop over surveys
    for surveyCounter in range(surveyCount):

        # Read current survey Id
        currentSurveyId = surveys.loc[surveyCounter, 'SurveyId']

        # Read questions of current survey Id
        questionsinCurrentSurvey = pd.read_sql(queryQuestionsinSurvey(currentSurveyId), sql_conn)
        questionCount = len(questionsinCurrentSurvey)

        # Initialize the output column list
        strColumnsQueryPart = ''

        # Loop over questions of current survey
        for questionCounter in range(questionCount):

            # Read InSurvey value
            currentInSurvey = questionsinCurrentSurvey.loc[questionCounter, 'InSurvey']

            # Read QuestionId values
            currentQuestionId = questionsinCurrentSurvey.loc[questionCounter, 'QuestionId']

            if currentInSurvey == 0:  # Current question is not in the current survey

                # The values in this column will be NULL
                strColumnsQueryPart += strQueryTemplateForNullColumnn(currentQuestionId)

            else:                    # currentInSurvey == 1 and current question is in the current survey

                # Use AnswerColumn query template and currentQuestionId to build a query for current column
                strColumnsQueryPart += strQueryTemplateForAnswerColumn(currentSurveyId, currentQuestionId)

            # Add a comma after each column except the last
            if questionCounter < questionCount-1:
                strColumnsQueryPart += ' , '

        # Build query to fetch current survey ID, the questions and answers, then append it to the final query
        strFinalQuery += strQueryTemplateOuterUnionQuery(currentSurveyId, strColumnsQueryPart)

        # Add UNION keyword after each subquery (returning current survey ID, the questions and answers) except the last
        if surveyCounter < surveyCount-1:
            strFinalQuery += ' UNION '
    
    return strFinalQuery

def RefreshView(View_Name, sql_conn):
    """This function is used to connect to database, execute the generated query : GetAllSurveyDataSQL
       and refresh the input view : View_Name"""
    
    query = ' CREATE OR ALTER VIEW '+View_Name+' AS '+GetAllSurveyDataSQL(sql_conn)
    
    # Get database cursor
    cursor = sql_conn.cursor()
    
    # Execute query
    cursor.execute(query)
    sql_conn.commit()
    
    cursor.close()

**MAIN FUNCTION**

In [73]:
def Always_Fresh(survey_struct_file):
    """Main function which reads a persisted version of survey structure table (in csv), compares it with
       survey structure table in SQL Server then updates the csv and refreshes the view : vw_GetAllSurveyData
       when necessary"""
    
    # Try connection to database
    try:
        sql_conn = pyodbc.connect(sql_conn_string, timeout=3)
        
        # Fetch the fresh survey structure from the database
        fresh_survey_structure = pd.read_sql(querySurveyStructure, sql_conn)

        # If the persisted survey structure does not exist, create it and update view : vw_GetAllSurveyData
        if not os.path.isfile(survey_struct_file):
            
            fresh_survey_structure.to_csv(survey_struct_file, index=False, sep=',')
            
            # Refresh View
            RefreshView('vw_GetAllSurveyData', sql_conn)
            
            print("The persistent survey structure has been created and view : vw_GetAllSurveyData created.")

        # If the persisted survey structure exists
        else:
            # Check if the persisted survey structure is identical to the fresh survey structure
            persisted_survey_structure = pd.read_csv(survey_struct_file, sep=',')

            # Persisted survey structure and database survey structure are different
            if not fresh_survey_structure.equals(persisted_survey_structure):

                # Update persisted survey structure
                fresh_survey_structure.to_csv(survey_struct_file, index=False, sep=',')

                # Refresh View
                RefreshView('vw_GetAllSurveyData', sql_conn)
                
                print("Changes in database survey structure table have been detected. The persistent survey structure has been successfully updated and the view : vw_GetAllSurveyData refreshed.")

            # Persisted survey structure and database survey structure are identical
            else:
                print("No changes detected in database survey structure table. No action performed.")
                pass
        
        # Close database connection
        sql_conn.close()
    
    # Unable to connect to database after 3s timeout
    except pyodbc.Error:
        print("Unable to connect to database. Please check the connection string.")         

**CALL MAIN FUNCTION**

In [76]:
# Call main function with path to persisted survey structure file
Always_Fresh('persistent_survey_structure.csv')

No changes detected in database survey structure table. No action performed
